In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle

import pandas as pd

import shap
from util.prediction import prepare_experiment_dfs

/home/ruptho/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df_crawled, df_reg, df_editreg, df_class = prepare_experiment_dfs('data/events/all_events.csv.gz')

In [ ]:
from util.shapley import get_indices, compute_shap_values
from util.prediction import load_rf_and_xgb_models

shap_vals_noreg, shap_vals_noreg_alt, shap_vals_noreg_alt_int = {}, {}, {}
models, model_eval = load_rf_and_xgb_models(df_class, df_reg, df_editreg)

In [ ]:
for key, model in model_eval.items():
    # SHAP values
    if '_xgb' in key:
        print(f'Retrain {key}')
        model.retrain_full()
        print(f'Fit shapval {key}')
        df_full = pd.concat([model.df_train_full, model.df_test])[model.encoded_columns]
        cat_indices = get_indices(model.encoded_columns, baselines={'code_en': 4, 'cat_disaster': 4})
        shap_vals_noreg[key] = compute_shap_values(model.full_model, df_full, df_full, model.encoded_columns,
                                                   cat_indices)
        explainer = shap.Explainer(model.full_model)
        shap_vals_noreg_alt[key] = explainer(df_full)
        shap_vals_noreg_alt_int[key] = explainer.shap_interaction_values(df_full)

        with open(f'shap/ACV_SHAP_SHAPINT_noreg.pkl', 'wb') as outp:  # Overwrites any existing file.
            pickle.dump([shap_vals_noreg, shap_vals_noreg_alt, shap_vals_noreg_alt_int], outp, pickle.HIGHEST_PROTOCOL)

for key, model in model_eval.items():
    # ACV shap values
    if '_xgb' in key:
        print(f'Retrain {key}')
        model.retrain_full()
        print(f'Fit shapval {key}')
        df_full = pd.concat([model.df_train_full, model.df_test])[model.encoded_columns]
        cat_indices = get_indices(model.encoded_columns, baselines={'code_en': 4, 'cat_disaster': 4})
        shap_vals_noreg[key] = compute_shap_values(model.full_model, df_full, df_full, model.encoded_columns,
                                                   cat_indices)

        with open(f'shap/ACV_SHAP_SHAPINT_noreg.pkl', 'wb') as outp:  # Overwrites any existing file.
            pickle.dump([shap_vals_noreg, shap_vals_noreg_alt, shap_vals_noreg_alt_int], outp, pickle.HIGHEST_PROTOCOL)